#### In this project we will scrape a table from Wikipedia with information of the Postal Codes, Boroughs and Neighbourhoods of the city. The final result will be a table with the Neighbourhoods grouped by Postal Codes.

First of all we will import and install the libraries and packages required, numpy, pandas, BeautifulSoup and requests.

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install BeautifulSoup4
from bs4 import BeautifulSoup
print('Libraries imported.')

     |████████████████████████████████| 102kB 18.9MB/s ta 0:00:01
Libraries imported.


In [3]:
!pip install lxml
!pip install requests

     |████████████████████████████████| 5.8MB 29.4MB/s eta 0:00:01


Using pandas the information in the website can be transformed into a Dataframe

In [4]:
import requests
link= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
df=pd.read_html(link)[0]
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Once the data frame is created a cleaning of the data is needed. Postal codes with no assigned Neighbourhoods will be dropped and Neighbourhoods will be grouped by Post Codes.

In [5]:
df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df_group=df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x))
df_group.reset_index(inplace=True)
df_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Once the cleaning of the dataframe is complete, we will see the shape of the result

In [6]:
df_group.shape

(103, 3)

Now the CSV containing latitude and longitude will be uploaded and transformed into a dataframe

In [7]:
df_coord=pd.read_csv('http://cocl.us/Geospatial_data')
df_coord.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_coord.head(5)
df_group_coord=pd.merge(df_group,df_coord[["Postcode", "Latitude", "Longitude"]], on="Postcode",how='left')
df_group_coord.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


We will import the packages needed for the rest of the project

In [8]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

The information for FourSquare

In [9]:
CLIENT_ID = 'FFFT5X3P0GB5WUVYEAJHMUDK31TBVTRRC4EFNG2AY25WHHA3' 
CLIENT_SECRET = 'SIR1KEBIXUNZMUTEYHVPU2SOV5CWUSROQB0RHQQ4PUFU0VMU' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FFFT5X3P0GB5WUVYEAJHMUDK31TBVTRRC4EFNG2AY25WHHA3
CLIENT_SECRET:SIR1KEBIXUNZMUTEYHVPU2SOV5CWUSROQB0RHQQ4PUFU0VMU


Now we will find the coordinates of Toronto using Geolocator tool

In [10]:
address ='Toronto, T.O.'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.72090385, -79.3771857338025.


In [11]:
LIMIT=100
RADIUS=500
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, RADIUS, LIMIT)

Now we will create a dataframe with Toronto's neighborhoods and their coordinates

In [23]:
#get coordinates for the neighborhoods of Toronto
neigh_coord=[]
for name in df['Neighbourhood']:
    address_neig = ('{},Toronto, ON'.format(name))
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address_neig, timeout=10)
    if location == None:
        latitude=0
        longitude=0
    else :
        latitude = location.latitude
        longitude = location.longitude
    neigh_coord.append([name,latitude,longitude])

In [24]:
neig_coord_df = pd.DataFrame(neigh_coord)
neig_coord_df.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']
print(neig_coord_df.shape)
neig_coord_df.head()

(211, 3)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Rouge,43.804930,-79.165837
1,Malvern,43.809196,-79.221701
2,Highland Creek,43.790117,-79.173334
3,Rouge Hill,43.780271,-79.130499
4,Port Union,43.775504,-79.134976


In next line we will drop the neighborhoods that we couldn't find it's coordinates

In [29]:
neig_coord_df=neig_coord_df.drop(neig_coord_df[neig_coord_df['Neighborhood Latitude']==0].index)

In [30]:
print(neig_coord_df.shape)

(200, 3)


We will create a function that gets the categories of the venues

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We will define a function to get nearby venues of the neighborhoods of Toronto

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues=getNearbyVenues(names=neig_coord_df['Neighborhood'],
                               latitudes=neig_coord_df['Neighborhood Latitude'],
                               longitudes=neig_coord_df['Neighborhood Longitude'])

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(5921, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge,43.804930,-79.165837,Dean Park,43.804364,-79.169159,Park
1,Rouge,43.804930,-79.165837,Paul's Breakfast & Burgers,43.803835,-79.169825,Fast Food Restaurant
2,Malvern,43.809196,-79.221701,Shoppers Drug Mart,43.809610,-79.222729,Pharmacy
3,Malvern,43.809196,-79.221701,Subway,43.806805,-79.222515,Sandwich Place
4,Malvern,43.809196,-79.221701,Pizza Hut,43.808326,-79.220616,Pizza Place


In [35]:
count_toronto_venues=toronto_venues.groupby('Neighborhood').count()
count_toronto_venues.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'],axis=1,inplace=True)
count_toronto_venues.head(10)

,Venue
Neighborhood,
Adelaide,100
Agincourt,14
Agincourt North,28
Albion Gardens,9
Alderwood,9
Bathurst Manor,4
Bathurst Quay,25
Bayview Village,11
Bedford Park,1


Now a dummy matrix with the Neighborhoods and each type of venue

In [40]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.set_index('Neighborhood',inplace=True)
toronto_onehot.reset_index(inplace=True)
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South A

In [41]:
toronto_onehot.shape

(5921, 329)

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South A

In [45]:
toronto_grouped.shape

(197, 329)

We define a function to find the most common venues of each neighborhood

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

TOP 10 Venues for each Neighborhood

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted___ = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted___['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted___.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted___.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Seafood Restaurant,American Restaurant,Italian Restaurant,Japanese Restaurant,Steakhouse
1,Agincourt,Chinese Restaurant,Cantonese Restaurant,Asian Restaurant,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hong Kong Restaurant,Train Station,Empanada Restaurant,Ethiopian Restaurant
2,Agincourt North,Ice Cream Shop,Chinese Restaurant,Bakery,Convenience Store,Spa,Bank,Taco Place,Liquor Store,Beer Store,Fried Chicken Joint
3,Albion Gardens,Grocery Store,Pizza Place,Fried Chicken Joint,Video Store,Pharmacy,Beer Store,Sandwich Place,Fast Food Restaurant,Hardware Store,Falafel Restaurant
4,Alderwood,Pizza Place,Gym,Dance Studio,Skating Rink,Sandwich Place,Pharmacy,Coffee Shop,Pub,Filipino Restaurant,Exhibit


## Cluster Neighborhoods

In [89]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [90]:
# add clustering labels
neighborhoods_venues_sorted___.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neig_coord_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted___.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rouge,43.804930,-79.165837,1.0,Fast Food Restaurant,Park,Zoo,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
1,Malvern,43.809196,-79.221701,0.0,Pizza Place,Pharmacy,Fast Food Restaurant,Convenience Store,Grocery Store,Gym / Fitness Center,Fried Chicken Joint,Skating Rink,Bubble Tea Shop,Sandwich Place
2,Highland Creek,43.790117,-79.173334,1.0,Pharmacy,Park,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,Rouge Hill,43.780271,-79.130499,0.0,Train Station,Bus Line,Zoo,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
4,Port Union,43.775504,-79.134976,1.0,Park,Martial Arts Dojo,Zoo,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


In [91]:
toronto_merged.dropna(axis=0,inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
#print(toronto_merged.dtypes)
toronto_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rouge,43.804930,-79.165837,1,Fast Food Restaurant,Park,Zoo,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
1,Malvern,43.809196,-79.221701,0,Pizza Place,Pharmacy,Fast Food Restaurant,Convenience Store,Grocery Store,Gym / Fitness Center,Fried Chicken Joint,Skating Rink,Bubble Tea Shop,Sandwich Place
2,Highland Creek,43.790117,-79.173334,1,Pharmacy,Park,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
3,Rouge Hill,43.780271,-79.130499,0,Train Station,Bus Line,Zoo,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
4,Port Union,43.775504,-79.134976,1,Park,Martial Arts Dojo,Zoo,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Studying Clusters

### Cluster 1

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Malvern,Pizza Place,Pharmacy,Fast Food Restaurant,Convenience Store,Grocery Store,Gym / Fitness Center,Fried Chicken Joint,Skating Rink,Bubble Tea Shop,Sandwich Place
3,Rouge Hill,Train Station,Bus Line,Zoo,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
5,Guildwood,Train Station,Storage Facility,Baseball Field,Zoo,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
6,Morningside,Coffee Shop,Sandwich Place,Park,Pharmacy,Discount Store,Fast Food Restaurant,Electronics Store,Convenience Store,Beer Store,Supermarket
7,West Hill,Pizza Place,Pharmacy,Grocery Store,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Food & Drink Shop,Smoothie Shop,Bank,Mexican Restaurant
8,Woburn,Fast Food Restaurant,Discount Store,Coffee Shop,Pizza Place,Beer Store,Bank,Sandwich Place,Taco Place,Gym,Toy / Game Store
9,Cedarbrae,Fast Food Restaurant,Coffee Shop,Gym,Pizza Place,Park,Toy / Game Store,Clothing Store,Beer Store,Grocery Store,Hardware Store
10,Scarborough Village,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Pub,Pharmacy,Gym,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
11,East Birchmount Park,Grocery Store,Gas Station,Intersection,Beer Store,Zoo,Festival,Event Space,Exhibit,Falafel Restaurant,Farm
13,Kennedy Park,Fast Food Restaurant,Chinese Restaurant,Asian Restaurant,Grocery Store,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm


### Cluster 2

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rouge,Fast Food Restaurant,Park,Zoo,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
2,Highland Creek,Pharmacy,Park,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Port Union,Park,Martial Arts Dojo,Zoo,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
12,Ionview,Metro Station,Deli / Bodega,Park,Zoo,Festival,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
20,Birch Cliff,Gym Pool,Baseball Field,Park,Gym,Flower Shop,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Food Court
28,Clarks Corners,Park,Convenience Store,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
36,Upper Rouge,Fast Food Restaurant,Park,Zoo,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
39,Henry Farm,Park,Tennis Court,Zoo,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
42,Silver Hills,Middle Eastern Restaurant,Park,Tennis Court,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
53,Bathurst Manor,Convenience Store,Playground,Baseball Field,Park,Zoo,Festival,Event Space,Exhibit,Falafel Restaurant,Farm


### Cluster 3

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Steeles East,Playground,Zoo,Festival,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
208,South Steeles,Playground,Zoo,Festival,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market


### Cluster 4

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Parkview Hill,Construction & Landscaping,Zoo,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### Cluster 5

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,York Mills,Wine Shop,Zoo,Festival,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
